In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import scipy.stats as stats

pd.set_option("mode.chained_assignment", None)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline as backend_inline

backend_inline.set_matplotlib_formats("svg")

# Uciml repo
from ucimlrepo import fetch_ucirepo

# Pytorch device specific configuration
# Pytorch Gpu Configuration for cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pytorch Gpu Configuration for dml devices
# import torch_directml

# device = torch_directml.device()

# Set default devices
torch.set_default_device(device)

In [3]:
# import the data
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = pd.read_csv(url, sep=";")
data = data[data["total sulfur dioxide"] < 200]  # drop a few outliers

In [ ]:
cols2zscore = data.keys()
cols2zscore = cols2zscore.drop("quality")

labels = data["quality"].values
input_labels = labels[0:10]
output_labels = labels[10:]

data[cols2zscore] = data[cols2zscore].apply(stats.zscore)

data[0, 10]

KeyError: (0, 10)

In [11]:
Z = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

print(Z[0:4], Z[4:])

[0, 1, 2, 3] [4, 5, 6, 7, 8, 9]
